In [1]:
import sys
dlc_dir = '/Users/annateruel/phd_code/'
if dlc_dir not in sys.path:
    sys.path.append(dlc_dir)


In [2]:
from dlc.analysis.time_roi import ROIDrawer
import pandas as pd
import dlc.analysis.time_roi as time_roi
import os

1. Drawing ROIS with ROIdrawer tool

In [4]:
roi_drawer = ROIDrawer(video_path='/Users/annateruel/Desktop/0_test2.avi',save_dir='/Users/annateruel/Desktop/videos/', num_rois=4)
r = roi_drawer.draw_rois()

Drawing ROIs for video: /Users/annateruel/Desktop/0_test2.avi


2. Time in ROIs

In [ ]:
video_dir = '/Users/annateruel/Desktop/videos/'

In [ ]:
roi_polygons = {}
for file in os.listdir(video_dir):
    if file.endswith('roi.h5'):
        roi_file_path = os.path.join(video_dir, file)
        print(f"Processing ROI file: {file}")

        try:
            roi_data = pd.read_hdf(roi_file_path)
            roi_groups = roi_data.groupby(['index', 'shape-type'])
            polygons, _ = time_roi.PolygonROI.extract_polygons(roi_groups)
            # Create the key by removing '_roi.h5' from the file name
            key = file.replace('_roi.h5', '')
            roi_polygons[key] = polygons  # Store polygons with the modified key
            print(f"Extracted polygons from {file}: {len(polygons)}")
        except Exception as e:
            print(f"Error processing {file}: {e}")

In [ ]:
scorer = 'DLC_resnet50_capsaicin_malesOct20shuffle1_200000'
bodypart = 'Neck'

In [ ]:
time_in_roi = time_roi.TimeinRoi(fps=15)
results_df = time_in_roi.time_in_rois_dir(directory=video_dir, rois=roi_polygons, scorer=scorer, body_part=bodypart)
results_df    